In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams
rcParams["figure.max_open_warning"] = False

In [ ]:
from evaluate_data import read_df
from crazyflie_description_py.parameters import N_BUFFER, FS, FFTSIZE
import itertools

exp_name = '2020_11_03_sweep'; start_idx = 50

all_frequencies = np.round(np.fft.rfftfreq(N_BUFFER, 1/FS), 0)
#print(list(all_frequencies))

df_results = pd.DataFrame(columns=[
    'degree', 
    'props',
    'snr',
    'motors',
    'source',
    'times',
    'spectrogram',
    'frequency_matrix',
    'strongest_freqs'
])

DEGREE_LIST = [90]
PROPS_LIST = [False, True]
SNR_LIST = [False, True]
MOTORS_LIST = [False, True]
SOURCE_LIST = ['sweep']

for params_values in itertools.product(DEGREE_LIST, PROPS_LIST, SNR_LIST, MOTORS_LIST, SOURCE_LIST):
    params_dict = dict(zip(['degree', 'props', 'snr', 'motors', 'source'], params_values))
    params_dict['exp_name'] = exp_name

    df, df_pos = read_df(**params_dict)
    df = df.iloc[start_idx:]
    
    frequency_matrix = np.zeros((len(all_frequencies), len(df)))
    i_col = 0
    for j, row in df.iterrows():
        # check if frequencies are sorted
        #if not np.all(np.diff(row.frequencies) >= 0):
            #print('not sorted:', params_dict)
        for f in row.frequencies:
            
            # find the closest frequency bin to chosen frequency
            idx = np.argmin(np.abs(f-all_frequencies))
            err = abs(all_frequencies[idx] - f)
            #if err >= 1e-1:
                #print(f'did not find {f} in bins. error:', err)
            frequency_matrix[idx, i_col] = 1.0 
        sum_ = np.sum(frequency_matrix[:, i_col])
        assert sum_ == FFTSIZE, sum_
        i_col += 1
        
    spectrogram = np.zeros((FFTSIZE, len(df)))
    strongest_freqs = np.zeros(len(df))
    i_col = 0
    for __, row in df.iterrows():
        signals_f = row.signals_f
        
        spectrogram[:, i_col] = np.sum(np.abs(signals_f), axis=0)
        
        max_idx = np.argmax(spectrogram[:, i_col])
        #if params_dict['snr']:
        #    assert max_idx == 0, max_idx
        strongest_freqs[i_col] = row.frequencies[max_idx]
        i_col += 1
    
    params_dict['spectrogram'] = spectrogram
    params_dict['strongest_freqs'] = strongest_freqs
    params_dict['frequency_matrix'] = frequency_matrix
    params_dict['times'] = (df.loc[:, 'audio_timestamp'].values - df.iloc[0].audio_timestamp) / 1e6
    df_results.loc[len(df_results), :] = params_dict
df_results.tail()

In [ ]:
shape = len(SNR_LIST)*len(PROPS_LIST), len(MOTORS_LIST)
fig, axs = plt.subplots(*shape, sharey=True, sharex=True)
fig.set_size_inches(shape[0]*5, shape[1]*5)

fig_spec, axs_spec = plt.subplots(*shape, sharey=True, sharex=True)
fig_spec.set_size_inches(shape[0]*5, shape[1]*5)

fig_scat, axs_scat = plt.subplots(*shape, sharex=True, sharey=True)
fig_scat.set_size_inches(shape[0]*5, shape[1]*5)
for j, motors in enumerate(MOTORS_LIST):
    axs[0, j].set_title(f"motors {motors}")
    axs_scat[0, j].set_title(f"motors {motors}")
    for i, (snr, props) in enumerate(itertools.product(SNR_LIST, PROPS_LIST)):
        
        filter_ = {
            "motors": motors,
            "snr": snr,
            "props": props
        }
        row = df_results.loc[(df_results.motors==motors) & (df_results.snr==snr) & (df_results.props==props), :]
        row = row.iloc[0]
        spectrogram = row.spectrogram
        spectrogram_wide = np.full((len(all_frequencies), spectrogram.shape[1]), np.nan)
        mask = np.where(row.frequency_matrix > 0)
        spectrogram_wide[mask[0], mask[1]] = spectrogram.flatten()
        
        times = row.times
        
        plot_mask = (all_frequencies < 7000) & (all_frequencies > 200)
        
        axs[i, j].pcolormesh(times, all_frequencies[plot_mask], np.log(spectrogram_wide[plot_mask]))
        axs[i, 0].set_ylabel(f"snr {snr} props {props}")
        
        axs_spec[i, j].pcolormesh(times, range(FFTSIZE), np.log(spectrogram))
        axs_spec[i, 0].set_ylabel(f"snr {snr} props {props}")
        
        axs_scat[i, j].scatter(times, row.strongest_freqs)
        axs_scat[i, 0].set_ylabel(f"snr {snr} props {props}")